# Computación evolutiva e inteligencia de enjambre
## Ejercicio 1
Implemente las estructuras de datos y algoritmos básicos para la solución de un problema mediante algoritmos genéticos. Pruebe estas rutinas y compare los resultados con un método de gradiente descendiente para buscar el mínimo global de las siguientes funciones:
1. $ f(x) = - x \sin(\sqrt{|x|}) $, con $ x \in [-512...512] $
2. $ f(x) = x + 5 \sin(3x) + 8 \cos (5 x)$, con $x \in \mathbb R $ en el intervalo $[0...20]$
3. $ f(x, y) = (x^2 + y^2)^{0.25} [\sin^2 (50 (x^2 + y^2)^{0.1}) + 1]$, con $x, y \in [-100...100]$

Podemos considerar la codificación de un número decimal entre -512 y 512 teniendo en cuenta enteros, de la siguiente manera:
1 bit para el signo (0 para negativos, 1 para positivos)
9 bits para representar los enteros

In [1]:
from matplotlib import pyplot as plt
import numpy as np

### Inicialización al azar

In [6]:
def initialize(n_ind, len):
    # n_ind: cantidad de individuos
    # len: cantidad de bits por cromosoma
    return np.random.randint(2, size = (n_ind, len))

initialize(4, 10)

array([[1, 1, 1, 1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [1, 0, 1, 1, 1, 1, 1, 0, 1, 0]])